In [1]:
#%autosave 0
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

Kurzfassung: 
Es sterben folgende Figuren:
1. Theon Greyjoy
2. Nightking
3. Alle Mormonts (Sir Jorah und das kleine Mädchen)
4. Alle White Walker und die ganzen Wiederkehrer
5. Barek Dondarion
6. Melissandre
7. Der schwermütige Ed (Leiter der Nachtwache)

# The Shunting Yard Algorithm (Operator Precedence Parsing)

The function $\texttt{toInt}(s)$ tries to convert the string $s$ to an integer.  If this works out, the integer is returned.  Otherwise, the string $s$ is returned unchanged.

In [2]:
def toInt(s):
    try:
        return int(s)   
    except ValueError:
        return s

In [3]:
toInt('123')

123

In [6]:
toInt('abc123')

'abc123'

The module `re` provides support for <a href='https://en.wikipedia.org/wiki/Regular_expression'>regular expressions</a>.  These are needed for
<em style="color:blue;">tokenizing</em> a string.

In [7]:
import re

The function $\texttt{tokenize}(s)$ takes a string $s$ representing an arithmetic expression and splits this string into a list of tokens.
The string `regExp` in the implementation below is interpreted as follows:

  - The `r` in front of the apostrophe `'` specifies that the regular expression is defined as a <em style="color:blue;">raw string</em>.  In a *raw string* the backslash does not have to be escaped because it is treated as a literal character.</br>  
  - The regular expression is divided into three parts. These parts are separated by the character `|`.  
      1. `[0-9]+` matches a natural number.  For example, it matches `0`or `123`.  It would also match a string like `007`.
         The `+` at the end of the substring `[0-9]*` specifies that there are any positive number of the characters in the range `[0-9]`.</br>
      2. `\*\*` matches the operator `**`.</br>
      3. `[()+*/%-]` matches a parenthesis or an arithmetical operator.  Note that the operator `-` has to be the last character in the 
         set of charaters specified inside the square brackets since otherwise it would be interpreted as a 
         <em style="color:blue;">range operator</em>.</br>

In [12]:
def tokenize(s):
    regExp = r'([0-9]+|\*\*|[()+*%/-])'
    L = [ toInt(t) for t in re.findall(regExp, s) ]
    return list(reversed(L))

In [13]:
tokenize('12+34*56/3-(17+2**4)')

[')', 4, '**', 2, '+', 17, '(', '-', 3, '/', 56, '*', 34, '+', 12]

The function $\texttt{evalBefore}(\texttt{o}_1, \texttt{o}_2)$ receives to strings representing artithmetical operators.  It returns `True` if the operator $\texttt{o}_1$ should be evaluated before the operator $\texttt{o}_2$ in an arithmetical expression of the form $a \;\texttt{o}_1\; b \;\texttt{o}_2\; c$.

In [14]:
def evalBefore(o1, o2):
    if o1 =='(':
        return False
    Precedence = { '+': 1, '-': 1, '*': 2, '/': 2, '%': 2, '**' : 3 }
    if Precedence[o1] > Precedence[o2]:
        return True
    elif Precedence[o1] == Precedence[o2]:
        if o1 == o2:
            return o1 in { '+', '-', '*', '/', '%' }
        else:
            return True
    else:
        return False

In [15]:
import stack

The class `Calculator` supports three member variables:
  - the token stack `mTokenStack` 
  - the operator stack `mOperators`
  - the argument stack `mArguments`
  
The constructor takes a string that is tokenized and pushes the tokens onto the token stack such that the first token is on top of the token stack.

In [16]:
class Calculator:
    def __init__(self, s):
        self.mTokens    = stack.createStack(tokenize(s))
        self.mOperators = stack.Stack()
        self.mArguments = stack.Stack()    

The method `__str__` is used to convert an object of class `Calculator` to a string.

In [17]:
def toString(self):
    return '\n'.join(['_'*50, 
                      'Tokens:    ' + str(self.mTokens), 
                      'Arguments: ' + str(self.mArguments), 
                      'Operators: ' + str(self.mOperators), 
                      '_'*50])

Calculator.__str__ = toString

The function $\texttt{evaluate}(\texttt{self})$ evaluates the expression that is given by the tokens on the `mTokenStack`.

In [18]:
def evaluate(self):
    while not self.mTokens.isEmpty():
        print(self)
        t = self.mTokens.top(); self.mTokens.pop()
        if isinstance(t, int):
            self.mArguments.push(t)
            continue
        if (self.mOperators.isEmpty() or t == "("):
            self.mOperators.push(t)
            continue
        topOp = self.mOperators.top()
        if topOp == "(" and t == ")":
            self.mOperators.pop()
        elif (t == ")" or evalBefore(topOp, t)):
            self.popAndEvaluate()
            self.mTokens.push(t)
        else:
            self.mOperators.push(t)
    while not self.mOperators.isEmpty():
        print(self)
        self.popAndEvaluate()
    return self.mArguments.top()
    
Calculator.evaluate = evaluate

The method $\texttt{popAndevaluate}(\texttt{self})$ removes the two topmost numbers $\texttt{rhs}$ and $\texttt{lhs}$ from the argument stack and 
removes the topmost operator $\texttt{op}$ from the argument stack.  It computes the value
$$ \texttt{lhs} \;\texttt{op}\; \texttt{rhs} $$
and pushes this value on the argument stack.

In [19]:
def popAndEvaluate(self):
    rhs = self.mArguments.top(); self.mArguments.pop()
    lhs = self.mArguments.top(); self.mArguments.pop()
    op  = self.mOperators.top(); self.mOperators.pop()
    result = None
    if op == '+':
        result = lhs + rhs
    if op == '-':
        result = lhs - rhs
    if op == '*':
        result = lhs * rhs
    if op == '/':
        result = lhs // rhs
    if op == '%':
        result = lhs % rhs
    if op == '**':
        result = lhs ** rhs
    assert result != None, f'ERROR: *** Unknown Operator *** "{op}"'
    self.mArguments.push(result)
    
Calculator.popAndEvaluate = popAndEvaluate

In [20]:
C = Calculator('1+2*3-4**2')

In [21]:
C.evaluate()

__________________________________________________
Tokens:    | 2 | ** | 4 | - | 3 | * | 2 | + | 1 |
Arguments: |
Operators: |
__________________________________________________
__________________________________________________
Tokens:    | 2 | ** | 4 | - | 3 | * | 2 | + |
Arguments: | 1 |
Operators: |
__________________________________________________
__________________________________________________
Tokens:    | 2 | ** | 4 | - | 3 | * | 2 |
Arguments: | 1 |
Operators: | + |
__________________________________________________
__________________________________________________
Tokens:    | 2 | ** | 4 | - | 3 | * |
Arguments: | 1 | 2 |
Operators: | + |
__________________________________________________
__________________________________________________
Tokens:    | 2 | ** | 4 | - | 3 |
Arguments: | 1 | 2 |
Operators: | + | * |
__________________________________________________
__________________________________________________
Tokens:    | 2 | ** | 4 | - |
Arguments: | 1 | 2 | 3 |
Oper

-9